# Create redshift summary `zall` value added catalog: Tile-Cumulative Version
Stéphanie Juneau (NOIRLab)

## Summary

Notebook used to create the `ztile` VAC (`zall-tilecumulative-edr-vac.fits`), which supersedes the redshift catalog `zall-tilecumulative-fuji.fits`. The EDR redshift VACs are described here: TODO: link to data doc website

### Steps
- Read in redshift catalog (`zall-tilecumulative-fuji.fits`)
- Make a list of all SURVEYs, PROGRAMs
- Iterate over each SURVEY-PROGRAM combination and create a file with the following coadded FIBERMAP quantities
    - MEAN_FIBER_X, Y, RA, DEC
    - STD_FIBER_RA, DEC
    - MEAN_DELTA_X, Y
    - RMS_DELTA_X, Y
    - MIN_, MEAN_, MAX_MJD
    - FIRSTNIGHT, LASTNIGHT
- Assemble the patch files together to update values of existing columns and add four new columns: MIN_, MEAN, MAX_MJD, FIRSTNIGHT
- Save the resulting VAC

*Important note*: The `LASTNIGHT` column already exists and referts to the last NIGHT (KPNO calendar date) during whcih the tile (TILEID) was observed regardless if some TARGETIDs might be missing due to hardware problems in early phases of the survey validation (occurred in SV1).

## Software

This notebook can be run with DESI 22.2, 22.5 or 23.1 kernels. However, it is important to note that it behaves the same as `desispec/0.59.0` in terms of the decisions made when coadding FIBERMAP information from individual exposures (see [desispec.coaddition.coadd_fibermap()](https://github.com/desihub/desispec/blob/0.59.0/py/desispec/coaddition.py))

## Imports

In [1]:
import os
import time
import numpy as np

from astropy.table import Table, join, vstack
from astropy.io import fits

# For FIBERSTATUS
from desispec.fiberbitmasking import get_all_fiberbitmask_with_amp, get_all_nonamp_fiberbitmask_val, get_justamps_fiberbitmask

## Initial Setup (NERSC or Astro Data Lab?)

In [2]:
platform = 'datalab'
#platform = 'nersc'

In [3]:
# NERSC is the primary platform to access DESI files (below for DESI members)
if platform=='nersc' :
    desi_dir = "/global/cfs/cdirs/desi/"
    
    # Ouput directory for VAC (tests/ copy to final location after vetting)
    vac_dir = desi_dir+"science/gqp/vac/edr/zcat/tests/"
    
    # Temporary directory for storing temporary patched coadded fibermap files
    tmp_dir = desi_dir+"science/gqp/vac/edr/zcat/tmp_patch/"
    
# At Data Lab, using relative directories (available to DESI members for testing upon request)
# (primary access to DESI EDR catalogs for users at Data Lab is via the databases)
if platform=='datalab' :
    
    desi_dir = "../../../../../DESI/"

    # Ouput directory for VAC
    vac_dir = "../../fuji/"
    
    # Temporary directory for storing temporary patched coadded fibermap files
    tmp_dir = "../../fuji/tmp_patch/"

## TILE coadd redshift catalog: `zall-tilecumulative`

In [4]:
# List all combinations for fuji
surveys_all = ['cmx', 'special',  'sv1',  'sv1',  'sv1',  'sv1',  'sv2',  'sv2',  'sv2',  'sv3',  'sv3',  'sv3']
programs_all = ['other', 'dark', 'backup', 'bright', 'dark', 'other', 'backup', 'bright', 'dark', 'backup', 'bright', 'dark']

N = len(surveys_all)

In [5]:
# Original zcatalog
zall_orig_file = desi_dir+"spectro/redux/fuji/zcatalog/zall-tilecumulative-fuji.fits"

# New zcat VAC
zvac_file = vac_dir+"zall-tilecumulative-edr-vac.fits"

## Coadded fibermap patch for each {survey}-{program}

This handles separately a problematic duplicated tile (`TILEID=80870`) with two different values of `LASTNIGHT`. This special consideration is not needed for `zpix`, only for `ztile`.

In [6]:
def coadd_fibermap_table(survey, program, fibermap_input, Ntruth):

    print(f"SURVEY={survey}; PROGRAM={program}")
    Nuniq = len(np.unique(fibermap_input['TARGETID','TILEID']))
    if (survey!='sv1' or program!='other') and Nuniq!=Ntruth:
        msg = "Incompatible number"
        raise ValueError(msg)
    
    # some cols get combined into mean or rms ('FIBER_X', 'FIBER_Y' for tile-coadds *only*)
    mean_cols = [
        'DELTA_X', 'DELTA_Y', 
        'FIBER_X', 'FIBER_Y',
        'PSF_TO_FIBER_SPECFLUX',
        'MJD']

    #- treat fiber coordinates separately due to bug w/ FIBER_RA==FIBER_DEC==0
    mean_coords = ['FIBER_RA', 'FIBER_DEC']

    #- rms_cols and std_cols must also be in mean_cols
    rms_cols = ['DELTA_X', 'DELTA_Y']
    std_cols = ['FIBER_RA', 'FIBER_DEC']
        
    #- Only a subset of "good" FIBERSTATUS flags are included in the coadd
    fiberstatus_nonamp_bits = get_all_nonamp_fiberbitmask_val()
    fiberstatus_amp_bits = get_justamps_fiberbitmask()
    
    #- check if any of the bad fiberstatus are set ignoring the amps for now
    # (also ignoring warnings such as RESTRICTED, STUCKPOSITIONER, POORPOSITION)
    nonamp_fiberstatus_flagged = ((fibermap_input['FIBERSTATUS'] & fiberstatus_nonamp_bits) > 0 )
    #- check is all 3 amps are bad as coadd(spectra) works on individual amp/arm so any good amp can be used
    allamps_flagged = ( (fibermap_input['FIBERSTATUS'] & fiberstatus_amp_bits) == fiberstatus_amp_bits )

    # Add column with whether to keep this entry in the COADD (Bool)
    fibermap_input['good_coadds'] = np.bitwise_not( nonamp_fiberstatus_flagged | allamps_flagged )    
 
    # Create grouped tables --> quantities for the good FIBERSTATUS given priority 
    # --> When they don't exist (all FIBERSTATUS are bad) quantities are computed over all rejects 
    # Solution: groupby('TARGETID','good_coadds') and inverse-order so that True comes before False   

    # Group joined table per TARGETID-TILEID (all rows)
    fmap_group_all = fibermap_input.group_by(['TARGETID','TILEID'])

    # Group joined table per TARGETID-TILEID and per good_coadds status
    fmap_group = fibermap_input.group_by(['TARGETID','TILEID','good_coadds'])
 
    #- mean cols (except for coordinate special case)
    t_mean = fmap_group[['TARGETID','TILEID','good_coadds']+mean_cols].groups.aggregate(np.mean)
    for col in ['FIBER_X','FIBER_Y','DELTA_X','DELTA_Y','PSF_TO_FIBER_SPECFLUX']: t_mean[col] = (t_mean[col]).astype(np.float32)
    t_mean.rename_columns(mean_cols, ['MEAN_'+col for col in mean_cols])

    #- min & max MJD cols
    t_min = fmap_group['TARGETID','TILEID','good_coadds','MJD'].groups.aggregate(np.min)
    t_min.rename_column('MJD', 'MIN_MJD')
    t_max = fmap_group['TARGETID','TILEID','good_coadds','MJD'].groups.aggregate(np.max)
    t_max.rename_column('MJD', 'MAX_MJD')

    #-first & last NIGHT cols (*not* using good_coadds condition to count all exposures)
    t_first = fmap_group_all['TARGETID','TILEID','NIGHT'].groups.aggregate(np.min)
    t_first.rename_column('NIGHT', 'FIRSTNIGHT')
    t_last = fmap_group_all['TARGETID','TILEID','NIGHT'].groups.aggregate(np.max)
    t_last.rename_column('NIGHT', 'LASTNIGHT')
    
    #- rms (root mean square) cols: rms = sqrt(mean(x**2))
    fmap_rms = fmap_group[['TARGETID','TILEID','good_coadds']+rms_cols]
    for col in rms_cols: fmap_rms[col] = fmap_rms[col]**2  #square
    t_rms = fmap_rms[['TARGETID','TILEID','good_coadds']+rms_cols].groups.aggregate(np.mean)  #mean
    for col in rms_cols: t_rms[col] = (np.sqrt(t_rms[col])).astype(np.float32)  #root
    
    t_rms.rename_columns(rms_cols, ['RMS_'+col for col in rms_cols])

    #- Combine results so far:
    t_join = t_mean
    t_join['SURVEY'] = survey
    t_join['PROGRAM'] = program
    t_join = join(t_join, t_rms, join_type='left', keys=['TARGETID','TILEID','good_coadds'])
    t_join = join(t_join, t_min, join_type='left', keys=['TARGETID','TILEID','good_coadds'])
    t_join = join(t_join, t_max, join_type='left', keys=['TARGETID','TILEID','good_coadds'])
    
    #- Sort on TARGETID and reverse of good_coadds (to keep True=1 before False=0)
    t_join.sort(keys=['TARGETID','TILEID','good_coadds'], reverse=True)
    u, indices = np.unique(t_join['TARGETID','TILEID'], return_index=True)
    t_uniq = t_join[indices]
    t_uniq.rename_column('good_coadds', 'COADD_EXIST')
    
    #- Check if can join also with first,last night
    if len(t_uniq)==len(t_first):
        t_uniq = join(t_uniq, t_first, join_type='left', keys=['TARGETID','TILEID'])
        t_uniq = join(t_uniq, t_last, join_type='left', keys=['TARGETID','TILEID'])
    else:
        msg = f'ERROR: Inconsistent number of rows in FIRST/LAST table: found {len(t_first)} but expecting {len(t_uniq)}'
        raise ValueError(msg)        
    
    #- Sanity check if reading a coadded extension (will treat special case SV1-other with duplicated tile below)
    if (survey!='sv1' or program!='other') and len(t_uniq)!=Ntruth:
        msg = f'ERROR: Inconsistent number of rows in MEAN+MIN+MAX table: found {len(t_uniq)} but expecting {Ntruth}'
        raise ValueError(msg)        

    # Identify cases with missing coords to group them separately
    missing_coord = (fibermap_input['FIBER_RA']==0.)&(fibermap_input['FIBER_DEC']==0.)
    valid_coord = ~missing_coord

    #- Group again without the missing FIBER_RA/DEC
    fmap_select = fibermap_input[valid_coord]
    fmap_valid = fmap_select.group_by(['TARGETID','TILEID','good_coadds'])
    
    #- Special case of averaging coordinates without missing FIBER_RA/DEC    
    t_mean_coords = fmap_valid[['TARGETID','TILEID','good_coadds']+mean_coords].groups.aggregate(np.mean)
    t_mean_coords.rename_columns(mean_coords, ['MEAN_'+col for col in mean_coords])

    #- std cols: std = sqrt(mean([x-mean(x)]**2))
    fmap_std = fmap_valid[['TARGETID','TILEID','good_coadds']+std_cols]
    mean_values = join(fmap_std, t_mean_coords, join_type='left', keys=['TARGETID','TILEID','good_coadds'])

    for col in std_cols: 
        fmap_std[col] = (fmap_std[col] - mean_values['MEAN_'+col])**2  #square of (x-mean(x))

    t_std_coords = fmap_std[['TARGETID','TILEID','good_coadds']+std_cols].groups.aggregate(np.mean)  #mean
    for col in std_cols: t_std_coords[col] = np.sqrt(t_std_coords[col])  #root

    t_std_coords.rename_columns(std_cols, ['STD_'+col for col in std_cols])

    # Convert from degrees to arcseconds with cos(dec) term
    t_std_coords['STD_FIBER_RA'] = (t_std_coords['STD_FIBER_RA']*3600.*np.cos(np.radians(t_std_coords['STD_FIBER_DEC']))).astype(np.float32)
    t_std_coords['STD_FIBER_DEC'] = (t_std_coords['STD_FIBER_DEC']*3600.).astype(np.float32)
    
    #- Group together new FIBER coords tables and create unique table again
    t_join_coords = join(t_mean_coords, t_std_coords, join_type='left', keys=['TARGETID','TILEID','good_coadds'])
    
    #- Sort on TARGETID and reverse of good_coadds (to keep True=1 before False=0)
    t_join_coords.sort(keys=['TARGETID','TILEID','good_coadds'], reverse=True)
    u, indices = np.unique(t_join_coords['TARGETID','TILEID'], return_index=True)
    t_uniq_coords = t_join_coords[indices]
    t_uniq_coords.remove_column('good_coadds')
    
    #- Sanity check
    if (survey!='sv1' or program!='other') and len(t_uniq_coords)!=Ntruth:
        msg = f'ERROR: Inconsistent number of rows in FIBER COORDS table: found {len(t_uniq_coords)} but expecting {Ntruth}'
        raise ValueError(msg)    
    
    #- Create output table by joining all the new columns so far
    t_out = join(t_uniq, t_uniq_coords, keys=['TARGETID','TILEID'])
    
    #- Add SURVEY, PROGRAM columns because these came from {survey}-{program} files
    t_out['SURVEY']=survey
    t_out['PROGRAM']=program

    return(t_out)

In [7]:
def fix_dup_tile(fm, cat):
        
    early_night = 20210512
    keep = fm['NIGHT']<=early_night
    fm = fm[keep]
    
    N_cat = len(cat)
    
    survey = 'sv1'
    program = 'other'
    
    t_out_dup = coadd_fibermap_table(survey, program, fm, N_cat)
    print(f"Returning a table with N(rows)={len(t_out_dup)}")
        
    return(t_out_dup)

In [8]:
# Loop over survey-program

for i in np.arange(N):

    # Start timer
    start_time = time.time()

    survey = surveys_all[i]
    program = programs_all[i]

    # Read file with two extensions per each survey-program
    zfile = f"{desi_dir}spectro/redux/fuji/zcatalog/ztile-{survey}-{program}-cumulative.fits"

    patch_file = f"{tmp_dir}ztile-{survey}-{program}-coaddpatch.fits"
    
    # By default, only generate the patch files once and skip if they exist (this is a time intensive step)
    if os.path.isfile(patch_file):
        continue
    
    print(f"Reading file: ztile-{survey}-{program}-cumulative.fits") 
    
    # Read the coadded extension just to keep the unique identifiers for testing/sanity checks
    zcat_single = Table.read(zfile, "ZCATALOG")
    zcat_truth = zcat_single['TARGETID','TILEID','LASTNIGHT']
    del zcat_single
    
    # Read the exp_fibermap to use for calculations
    fibermap_input = Table.read(zfile, "EXP_FIBERMAP")
    
    # Keep only rows with TARGETID>0
    zcat_truth = zcat_truth[zcat_truth['TARGETID']>0]
    fibermap_input = fibermap_input[fibermap_input['TARGETID']>0]

    # Number of expected rows to cross-check later
    Ntruth = len(zcat_truth)
    
    #- Call the coadd_fibermap_table() function
    t_out = coadd_fibermap_table(survey, program, fibermap_input, Ntruth)
    
    #- Sanity check for all regular cases
    if (survey!='sv1' or program!='other') and len(t_out)!=Ntruth:
        msg = f'ERROR: Inconsistent number of rows in OUTPUT table: found {len(t_out)} but expecting {Ntruth}'
        raise ValueError(msg)

    #- Sanity check and dealing with the exception duplicate tile
    if len(t_out)!=Ntruth and survey=='sv1' and program=='other' :
        # Tile with Duplicate LASTNIGHT
        duptile = 80870
        fm_dup = fibermap_input[fibermap_input['TILEID']==duptile]
        cat_dup = t_out[t_out['TILEID']==duptile]
            
        print(f"BEFORE the fix = {len(t_out)}")
            
        # Fix the known issue with duplicate TILEID in sv1-other with supplement for extra LASTNIGHT
        t_out_dup = fix_dup_tile(fm_dup, cat_dup)
        print(f"... supplement table = {len(t_out_dup)}")
             
        t_out = vstack([t_out, t_out_dup])
        print(f"AFTER the fix = {len(t_out)}")
            
        # Check again to make sure the fix worked
        if len(t_out)!=Ntruth:
            msg = f'ERROR: Inconsistent number of rows in OUTPUT table for SV1-other: found {len(t_out)} but expecting {Ntruth}'
            raise ValueError(msg)
        
    #- Save output file per {survey}-{program}
    t_out.write(patch_file, overwrite=True)
    print(f"Wrote output file: {patch_file}")
   
    # End timer
    end_time = time.time()

    # Calculate elapsed time
    elapsed_time = np.round(end_time - start_time, 2)
    print("----------------- Elapsed time [sec]: ", elapsed_time)   

In [9]:
# Free memory (only if actually running the patch files)
#del fibermap_input
#del t_out

## Original zall file

In [10]:
fits.info(zall_orig_file)

Filename: ../../../../../DESI/spectro/redux/fuji/zcatalog/zall-tilecumulative-fuji.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  ZCATALOG      1 BinTableHDU    357   3611000R x 141C   [K, 7A, 6A, J, J, D, D, K, D, 10D, K, 6A, 20A, K, D, I, J, K, J, J, D, D, E, E, E, E, K, B, 3A, E, E, J, D, J, I, 8A, J, J, 4A, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, I, E, E, E, E, K, 2A, E, E, E, E, 1A, K, K, K, K, K, K, K, K, K, K, K, K, K, K, K, K, K, K, K, D, D, J, I, E, I, I, E, E, E, E, D, E, D, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, J, L, K, L]   


In [11]:
%%time
# Read input catalog and only keep cases with TARGETID>0
tz = Table.read(zall_orig_file)
tz = tz[tz['TARGETID']>0]

CPU times: user 12.1 s, sys: 2.55 s, total: 14.6 s
Wall time: 14.6 s


## Read in patch files

In [12]:
def find_dup_tile(arr_in):
    tt = Table(arr_in)
    print(f"Table with N rows = {len(tt)}")
    
    u, indices, counts = np.unique(tt['TILEID'],return_index=True, return_counts=True)
    checktile = u[counts>1]
    
    out = tt[np.isin(tt['TILEID'], checktile)]
    print(f"Num rows with duplicated tiles = {len(out)}")
    
    for tile in out['TILEID']:
        print(f"Tile {tile} has LASTNIGHT=", out['LASTNIGHT'][out['TILEID']==tile])
        
    return(out)

In [13]:
t_patch = Table()

# N
for i in np.arange(N):
    survey = surveys_all[i]
    program = programs_all[i]

    # Read file per each survey-program
    filename = f"{tmp_dir}ztile-{survey}-{program}-coaddpatch.fits"
    t = Table.read(filename)
    
    t_patch = vstack([t_patch, t])
    
    print("=============================================================")
    print(f"SURVEY={survey}; PROGRAM={program}")
    print("---- for patch ----")
    print(len(np.unique(t['TARGETID','TILEID'])))
    print(len(np.unique(t['TILEID'])))
    print(len(np.unique(t['TILEID','LASTNIGHT'])))
#    check_cases = find_dup_tile(np.unique(t['TILEID','LASTNIGHT']))
    
    print("---- for original ----")
    tz_sel = tz[(tz['SURVEY']==survey)&(tz['PROGRAM']==program)]
    print(len(np.unique(tz_sel['TARGETID','TILEID','LASTNIGHT'])))
    print(len(np.unique(tz_sel['TARGETID','TILEID'])))
    print(len(np.unique(tz_sel['TILEID'])))
    print(len(np.unique(tz_sel['TILEID','LASTNIGHT'])))
    

SURVEY=cmx; PROGRAM=other
---- for patch ----
5000
1
1
---- for original ----
5000
5000
1
1
SURVEY=special; PROGRAM=dark
---- for patch ----
67201
16
16
---- for original ----
67201
67201
16
16
SURVEY=sv1; PROGRAM=backup
---- for patch ----
113845
26
27
---- for original ----
113845
113845
26
26
SURVEY=sv1; PROGRAM=bright
---- for patch ----
239500
50
76
---- for original ----
239500
239500
50
50
SURVEY=sv1; PROGRAM=dark
---- for patch ----
384000
80
95
---- for original ----
384000
384000
80
80
SURVEY=sv1; PROGRAM=other
---- for patch ----
146304
32
44
---- for original ----
151304
146304
32
33
SURVEY=sv2; PROGRAM=backup
---- for patch ----
4208
1
1
---- for original ----
4208
4208
1
1
SURVEY=sv2; PROGRAM=bright
---- for patch ----
70337
18
18
---- for original ----
70337
70337
18
18
SURVEY=sv2; PROGRAM=dark
---- for patch ----
84065
20
20
---- for original ----
84065
84065
20
20
SURVEY=sv3; PROGRAM=backup
---- for patch ----
152740
35
35
---- for original ----
152740
152740
35
35
SUR

In [14]:
# For info / debugging
#t_patch.colnames

In [15]:
print(len(t_patch))
t_patch[:5]

3207569


TARGETID,TILEID,COADD_EXIST,MEAN_DELTA_X,MEAN_DELTA_Y,MEAN_FIBER_X,MEAN_FIBER_Y,MEAN_PSF_TO_FIBER_SPECFLUX,MEAN_MJD,SURVEY,PROGRAM,RMS_DELTA_X,RMS_DELTA_Y,MIN_MJD,MAX_MJD,FIRSTNIGHT,LASTNIGHT,MEAN_FIBER_RA,MEAN_FIBER_DEC,STD_FIBER_RA,STD_FIBER_DEC
int64,int32,bool,float32,float32,float32,float32,float32,float64,bytes7,bytes6,float32,float32,float64,float64,int32,int32,float64,float64,float32,float32
39628473198708395,80615,False,0.0,0.0,0.0,0.0,0.7702122,59200.095110125,cmx,other,0.0,0.0,59200.06640136,59200.12381137,20201216,20201216,23.6619676773673,29.8475887928968,0.0,0.0
39628473198709499,80615,True,-0.0055,-0.00375,69.6375,-391.3065,0.7471383,59200.095110125,cmx,other,0.008093207,0.0124197425,59200.06640136,59200.12381137,20201216,20201216,23.711789506441093,29.843712547212316,0.1003415,0.15683237
39628473198710139,80615,True,-0.0085,-0.0035,62.71825,-382.719,0.7309045,59200.095110125,cmx,other,0.009949874,0.01317194,59200.06640136,59200.12381137,20201216,20201216,23.742668270905856,29.874927385824513,0.0867613,0.16850825
39628473198710603,80615,True,-0.00675,-0.0055,58.0,-394.30325,0.71950334,59200.095110125,cmx,other,0.0091515025,0.014124447,59200.06640136,59200.12381137,20201216,20201216,23.764893506253426,29.832358617984326,0.10309766,0.1715876
39628473198711006,80615,True,-0.002,-0.00875,53.11,-384.87424,0.789,59200.095110125,cmx,other,0.0068920245,0.01566046,59200.06640136,59200.12381137,20201216,20201216,23.786503075810064,29.866703322515196,0.10997444,0.17224741


In [16]:
print(f"N rows in patch table = {len(t_patch)}")
print(f"N with unqiue TARGETID,TILEID = {len(np.unique(t_patch['TARGETID','TILEID']))}")
print(f"N unique TILEIDs = {len(np.unique(t_patch['TILEID']))}")
print(f"N unique TILEID-LASTNIGHT = {len(np.unique(t_patch['TILEID','LASTNIGHT']))}")

N rows in patch table = 3207569
N with unqiue TARGETID,TILEID = 3202569
N unique TILEIDs = 732
N unique TILEID-LASTNIGHT = 787


In [17]:
print(f"N rows in zpix table = {len(tz)}")
print(f"N with unqiue TARGETID,TILEID = {len(np.unique(tz['TARGETID','TILEID']))}")
print(f"N with unqiue TARGETID,TILEID, LASTNIGHT = {len(np.unique(tz['TARGETID','TILEID','LASTNIGHT']))}")
print(f"N unique TILEIDs = {len(np.unique(tz['TILEID']))}")
print(f"N unique TILEID-LASTNIGHT = {len(np.unique(tz['TILEID','LASTNIGHT']))}")

N rows in zpix table = 3207569
N with unqiue TARGETID,TILEID = 3202569
N with unqiue TARGETID,TILEID, LASTNIGHT = 3207569
N unique TILEIDs = 732
N unique TILEID-LASTNIGHT = 733


## Replace and add columns based on patch files

In [18]:
%%time

# Check that lenghts and row order are consistent
Nz = len(tz)
Np = len(t_patch)

# Columns exist but values are updated
cols_to_replace = ['MEAN_FIBER_X', 'MEAN_FIBER_Y','MEAN_DELTA_X','RMS_DELTA_X','MEAN_DELTA_Y','RMS_DELTA_Y',\
                   'MEAN_FIBER_RA', 'STD_FIBER_RA', 'MEAN_FIBER_DEC', 'STD_FIBER_DEC','MEAN_PSF_TO_FIBER_SPECFLUX']

# LASTNIGHT already exists in the zall-tilecumulative file (leaving it as is)
cols_to_add = ['MIN_MJD', 'MEAN_MJD', 'MAX_MJD', 'FIRSTNIGHT']
cols_to_fix = ['LASTNIGHT']


if Nz==Np:
    tz.sort(['SURVEY','PROGRAM','TILEID','TARGETID'])
    t_patch.sort(['SURVEY','PROGRAM','TILEID','TARGETID'])
    
    # Fix LASTNIGHT to be able to use it properly
    duptile = 80870
    rfix_z = tz['TILEID']!=duptile
    rfix_patch = t_patch['TILEID']!=duptile

    t_patch['LASTNIGHT'][rfix_patch] = tz['LASTNIGHT'][rfix_z]

    # Check that this worked
    Nz_check = len(np.unique(tz['TARGETID','TILEID','LASTNIGHT']))
    Npatch_check = len(np.unique(t_patch['TARGETID','TILEID','LASTNIGHT']))
    if Nz_check!=Npatch_check:
        msg = (f"Uh oh the LASTNIGHT fix didn't work: got {Npatch_check} instead of {Nz_check}")
        raise ValueError(msg)
    
    # Re-sort with LASTNIGHT to take care of dup tile
    tz.sort(['SURVEY','PROGRAM','TILEID','LASTNIGHT','TARGETID'])
    t_patch.sort(['SURVEY','PROGRAM','TILEID','LASTNIGHT','TARGETID'])
    
    # Replace columns listed above
    for col in cols_to_replace:
        tz[col] = t_patch[col]

    # Add new columns listed above
    for col in cols_to_add:
        tz[col] = t_patch[col]
else:
    print(f"Different numbers: expecting {Nz} but the patch table has {Np}")    

CPU times: user 1min 15s, sys: 1.31 s, total: 1min 17s
Wall time: 1min 16s


In [19]:
print(len(tz))

# Check on columns that had problems in the past due to missing coordinates (now should be ignored)
finite_cols = ['MEAN_FIBER_RA', 'MEAN_FIBER_DEC', 'STD_FIBER_RA', 'STD_FIBER_DEC']
for col in finite_cols:
    print(len(tz[np.isfinite(tz[col])]))

3207569
3207569
3207569
3207569
3207569


## Save output file

In [ ]:
%%time
# Will need to add overwirte=True if wants to overwrite an existing file
tz.write(zvac_file)